In [3]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim([0, 10])
ax.set_ylim([0, 10])

coords = []

def onclick(event):
    plt.scatter(event.xdata, event.ydata)
    fig.canvas.draw()
    global ix, iy
    ix, iy = event.xdata, event.ydata
    
    global coords
    coords.append((ix, iy))

    if len(coords) == 15:
        fig.canvas.mpl_disconnect(cid)

    return coords
cid = fig.canvas.mpl_connect('button_press_event', onclick)


In [4]:
print(coords)

[(1.411290322580645, 2.0346320346320352), (0.7258064516129035, 3.630952380952382), (2.2580645161290325, 4.686147186147187), (4.133064516129032, 7.121212121212121), (9.193548387096774, 2.765151515151516), (6.330645161290322, 1.2229437229437234), (6.169354838709678, 3.4956709956709964), (4.959677419354838, 0.952380952380953), (2.7419354838709675, 0.6277056277056281), (3.0040322580645165, 2.927489177489178)]


In [ ]:
fig.canvas.mpl_disconnect(cid)

In [4]:
coords = np.array(coords)
coords = np.vstack((coords,coords[0,:] ))

# Function from https://towardsdatascience.com/b%C3%A9zier-interpolation-8033e9a262c2

# find the a & b points
def get_bezier_coef(points):
    # since the formulas work given that we have n+1 points
    # then n must be this:
    n = len(points) - 1

    # build coefficents matrix
    C = 4 * np.identity(n)
    np.fill_diagonal(C[1:], 1)
    np.fill_diagonal(C[:, 1:], 1)
    C[0, 0] = 2
    C[n - 1, n - 1] = 7
    C[n - 1, n - 2] = 2

    # build points vector
    P = [2 * (2 * points[i] + points[i + 1]) for i in range(n)]
    P[0] = points[0] + 2 * points[1]
    P[n - 1] = 8 * points[n - 1] + points[n]

    # solve system, find a & b
    A = np.linalg.solve(C, P)
    B = [0] * n
    for i in range(n - 1):
        B[i] = 2 * points[i + 1] - A[i + 1]
    B[n - 1] = (A[n - 1] + points[n]) / 2

    return A, B

# returns the general Bezier cubic formula given 4 control points
def get_cubic(a, b, c, d):
    return lambda t: np.power(1 - t, 3) * a + 3 * np.power(1 - t, 2) * t * b + 3 * (1 - t) * np.power(t, 2) * c + np.power(t, 3) * d

# return one cubic curve for each consecutive points
def get_bezier_cubic(points):
    A, B = get_bezier_coef(points)
    return [
        get_cubic(points[i], A[i], B[i], points[i + 1])
        for i in range(len(points) - 1)
    ]

# evalute each cubic curve on the range [0, 1] sliced in n points
def evaluate_bezier(points, n):
    curves = get_bezier_cubic(points)
    return np.array([fun(t) for fun in curves for t in np.linspace(0, 1, n)])

# generate 5 (or any number that you want) random points that we want to fit (or set them youreself)
points = np.random.rand(5, 2)

# fit the points with Bezier interpolation
# use 50 points between each consecutive points to draw the curve
path = evaluate_bezier(coords, 50)

# extract x & y coordinates of points
x, y = points[:,0], points[:,1]
px, py = path[:,0], path[:,1]

''' '''
# plot
plt.figure(figsize=(11, 8))
plt.plot(px, py, 'b-')
plt.show()